* 역전파는 순전파의 반대로 생각하면 편하다.
* 역전파를 이용해 미분값을 구할 때 덧셈인지 곱셈인지에 따라 미분방식이 달라진다.

In [1]:
# 순전파와 역전파의 메소드를 모두 가지고 있는 클레스 하나를 구현

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self,dout):
        dx = dout*self.y
        dy = dout*self.x
        return dx, dy

In [2]:
# 순전파 구현하기


# edge값 지정
apple = 100
apple_num = 2
tax = 1.1

# 계층들
apple_layer = MulLayer()
tax_layer = MulLayer()

# 순전파
apple_price = apple_layer.forward(apple,apple_num)
price = tax_layer.forward(apple_price,tax)

print(int(price))

220


In [3]:
# 역전파 구현하기
# x와 y는 클래스 객체를 생성하였을때 저장되어져 있다.

# 역전파
dprice = 1 # 초기값 지정
dapple_price,dtax = tax_layer.backward(dprice)
dapple,dapple_num = apple_layer.backward(dapple_price)

print(dapple,dapple_price,dapple_num)

2.2 1.1 110.00000000000001


In [4]:
# 덧셈 계층을 구현

class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out = x+y
        
        return out
    
    def backward(self,dout):
        dx = dout*1
        dy = dout*1
        return dx,dy

In [5]:
# 순전파 구현하기

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들

apple_layer = MulLayer()  # apple의 가격과 개수는 곱하는 것이기 때문에 곱셈 계층객체를 지정한다.
orange_layer = MulLayer() 

add_apple_orange_layer = AddLayer() 
tax_layer = MulLayer()

# 순전파 구현하기

apple_price = apple_layer.forward(apple,apple_num)  # 사과의 가격과 사과의 개수 곱하기
orange_price = orange_layer.forward(orange,orange_num) # 오렌지의 가격과 오렌지의 개수 곱하기
all_price = add_apple_orange_layer.forward(apple_price,orange_price) # 위의 두개의 가격 더하기
price = tax_layer.forward(all_price,tax) # 전체 가격에서 세금을 곱하기

print(int(price))

715


In [6]:
# 역전파 구현하기
# 순전파를 구현할때 객체들을 이미 생성했기 때문에 x,y들이 저장되어 있는 겍체들을 그대로 가져간다.

dprice =1
dall_price,d_tax = tax_layer.backward(dprice)
dapple_price,dorange_price = add_apple_orange_layer.backward(dall_price)
dorange,dorange_num = orange_layer.backward(dorange_price)
dapple,dapple_num = apple_layer.backward(dapple_price)

print(list(map(int,[d_tax,dorange,dorange_num,dapple,dapple_num]))) # 리스트 형태의 값들을 정수형으로 형변환시켜주기

[650, 3, 165, 2, 110]


# 계산 그래프를 신경망에 적용

In [7]:
# 활성화 함수 구현해보기
# 활성화 함수 : Relu , sigmoid

In [8]:
# Relu 계층 클래스 만들기
# 순전파 => x>0 이면 x 출력 , x<=0이면 0 출력
# 역전파 => x>0 이면 1 (그대로)출력 x<=0이면 신호를 보내지 않음

class Relu:
    
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0  #  out[self.mask]는 X<=0인 값들만을 추려내고 그 값들을 0으로 만든다.
        
        return out
        
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout   # x>0 이면 그대로
        
        return dx

In [9]:
import numpy as np
x = np.array([[1.0,-0.5],[-2.0,3.0]])
mask = x<=0
out = x.copy()

out[mask]

array([-0.5, -2. ])

In [10]:
# 시그모이드 계층 클래스 만들기

class sigmoid:
    
    def __init__(self):
        self.out = None
        
    def forward(self,x):     # 시그모이드 함수의 출력값 
        out = 1/(1+np.exp(-x))  # 시그모이드의 순전파는 x만을 가지고 y를 출력할 수 있다.
        self.out = out
        
        return out
    
    def backward(self,dout): # 시그모이드 역전파는 y만을 가지고 구할 수 있다.
        dx = dout * (1.0-self.out)*self.out
        
        return dx

In [11]:
# 신경망 복습 (입력층(x),가중치(w),편향(b))

x = np.random.rand(2)
w = np.random.rand(2,3)
b = np.random.rand(3)

y = np.dot(x,w) + b
print('출력값 : ',y)

# y를 활성화 함수를 통해 변환한다.(ex) 활성화함수 : 시그모이드
def sigmoid(y):
    return 1/(1+np.exp(-y))

print('활성화 함수 사용후 변환값 : ',sigmoid(y))

출력값 :  [0.51174735 0.50587036 0.09377255]
활성화 함수 사용후 변환값 :  [0.62521601 0.62383789 0.52342597]


In [12]:
# affine 계층 구현하기

class Affine:
    
    def __init__(self,w,b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.w) + self.b
        
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.w.T)
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        
        return dx

In [13]:
# 소프트맥스와 교차 엔트로피 계층 구현하기!!

class SoftmaxWithLoss:
    
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        
        return self.loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t)/batch_size
        
        return dx

# 오차 역전파법 구현하기!

* 신경망 학습의 전체 과정
* 1단계 : 미니배치
* 2단계 : 기울기 산출
* 3단계 : 매개변수 갱신
* 4단계 : 반복 (1~3단계)
    

* 오차역전파법은 2단계에서 등장한다.
* 수치미분을 통해 기울기를 구하는 방법보다 오차역전파법을 통해 기울기를 구하는 방법이 더 효율적이다.

In [18]:
import numpy as np
import sys, os
os.chdir('C:/Users/174518/Desktop/c언어,파이선/파이썬 데이터/밑바닥부터시작하는딥러닝1/deep-learning-from-scratch-master')
sys.path.append(os.pardir)

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict  # 딕셔너리에 추가한 순서를 기억한다.

class TwoLayerNet:
    
    def __init__(self,input_size, hidden_size, output_size, 
                weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.parmas['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()  # 추가한 순서 기억
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])  # affine 계층
        self.layers['Relu1'] = Relu()  # relu 계층
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss() # 출력값에 활성화 함수인 소프트맥스를 활용한뒤 교차엔트로피(손실함수)까지 구한다.
        
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)  # 행별 가장 큰 값의 위치 출력 ---> y는 one_hot_encoder 형태로 되어있다.
        if t.ndim != 1: t = np.argmax(t,axis=1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self,x,t):   # 수치미분을 이용해 손실함수의 기울기 구하기
        loss_W = lambda W : self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])  # 손실함수(loss)의 기울기 구하기
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])
        
        return grads
    
    def gradient(self,x,t):   # 역전파를 이용해 손실함수의 기울기 구하기
        # 순전파
        self.loss(x,t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
     
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

* 수치 미분은 속도가 느리기 때문에 신경망 학습에서는 오차역전파를 이용해 기울기를 구한다.
* 수치미분은 구현하기가 쉽기 때문에 오차역전파를 이용해 구현했을 때 기울기가 같은지 확인용도로 사용하게 된다.

In [21]:
# 기울기 확인!!   ----> 수치미분과 오차역전파 비교

import numpy as np
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet    # ch05폴더의 two_layer_net 파일에서 TwoLayerNet 클래스를 불러드린다.
                                               # 현재 폴더위치를 알고 있어야 한다.
    
# 데이터 읽기
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size= 50, output_size=10)

# 미니배치
x_batch = x_train[:3]
t_batch = t_train[:3]
                                               
# 수치미분을 이용한 기울기와 오차역전파를 이용한 기울기 구하기
grad_numerical = network.numerical_gradient(x_batch,t_batch)
grad_backprop = network.gradient(x_batch,t_batch)



In [32]:
for key in grad_numerical.keys():
     print(key+' : '+str(np.average(np.abs(grad_backprop[key] - grad_numerical[key]))))

W1 : 1.53411332319078e-07
b1 : 1.9317701362405264e-06
W2 : 4.4728863917642556e-09
b2 : 1.402873274491956e-07


In [33]:
print('수치미분한 기울기 : ',grad_numerical)
print('오차역전파한 기울기 : ',grad_backprop)

수치미분한 기울기 :  {'W1': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'b1': array([ 4.65716288e-05,  0.00000000e+00, -1.39972883e-03, -4.03743227e-03,
       -1.57002460e-03,  2.26777296e-03, -1.07096638e-03,  4.57507447e-03,
       -4.43162628e-03, -1.31679420e-03, -3.34194012e-03,  5.09493493e-03,
        5.80904537e-03,  0.00000000e+00, -7.87152076e-03,  0.00000000e+00,
        1.79814184e-03,  0.00000000e+00,  7.33851198e-03,  3.26773052e-03,
       -1.78407091e-04, -2.14584328e-03,  2.65828903e-03, -2.58667907e-03,
        8.04512148e-03, -5.99025644e-03,  2.70664734e-03,  4.52187829e-03,
       -9.12063327e-04,  0.00000000e+00,  4.76762768e-03, -2.11351386e-03,
       -3.87001491e-03,  0.00000000e+00,  4.61147346e-03,  2.33641898e-03,
        6.36588415e-04,  0.00000000e+00, -1.98645238e-03,